In [1]:
import torch
# from vit_pytorch import ViT
import torch.nn as nn
import torchvision.models as models
from torchsummary import summary
from einops.layers.torch import Rearrange, Reduce

from tqdm import tqdm
import albumentations as A

from albumentations.pytorch import ToTensorV2

/home/jo/anaconda3/envs/build/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jo/anaconda3/envs/build/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## utils

In [2]:
import random
import numpy as np
from typing import List, Union
from joblib import Parallel, delayed
import os
import torch.nn.functional as F

In [6]:
# 시드 고정 함수
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

# RLE 디코딩 함수
def rle_decode(mask_rle: Union[str, int], shape=(224, 224)) -> np.array:
    '''
    mask_rle: run-length as string formatted (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if mask_rle == -1:
        return np.zeros(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# dice score 계산 함수
def dice_score(prediction: np.array, ground_truth: np.array, smooth=1e-7) -> float:
    '''
    Calculate Dice Score between two binary masks.
    '''
    intersection = np.sum(prediction * ground_truth)
    return (2.0 * intersection + smooth) / (np.sum(prediction) + np.sum(ground_truth) + smooth)

def calculate_dice_scores(validation_df, img_shape=(224, 224)) -> List[float]:
    '''
    Calculate Dice scores for a dataset.
    '''
    # Extract the mask_rle columns
    pred_mask_rle = validation_df.iloc[:, 3]
    gt_mask_rle = validation_df.iloc[:, 4]

    def calculate_dice(pred_rle, gt_rle):
        pred_mask = rle_decode(pred_rle, img_shape)
        gt_mask = rle_decode(gt_rle, img_shape)
        if np.sum(gt_mask) > 0 or np.sum(pred_mask) > 0:
            return dice_score(pred_mask, gt_mask)
        else:
            return None  # No valid masks found, return None
    dice_scores = Parallel(n_jobs=-1)(
        delayed(calculate_dice)(pred_rle, gt_rle) for pred_rle, gt_rle in zip(pred_mask_rle, gt_mask_rle)
    )
    dice_scores = [score for score in dice_scores if score is not None]  # Exclude None values
    return np.mean(dice_scores)

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice
# class F_BetaLoss(nn.Module):
#     def __init__(self,  beta=1.0):
#         self.eps = 1e-5
#         self.beta = beta
#         super(F_BetaLoss, self).__init__()
#
#     def forward(self, inputs, targets):
#
#         #comment out if your model contains a sigmoid or equivalent activation layer
#         inputs = F.sigmoid(inputs)
#         #flatten label and prediction tensor
#         inputs = inputs.view(-1)
#         targets = targets.view(-1)
#         tp = torch.sum(inputs * targets)  # TP
#         fp = torch.sum(inputs * (1 - targets))  # FP
#         fn = torch.sum((1 - inputs) * targets)  # FN
#         # tn = torch.sum((1 - inputs) * (1 - targets))  # TN
#         precision = (tp + self.eps) / (tp + fp + self.eps)
#         recall = (tp + self.eps) / (tp + fn + self.eps)
#         f_b = (1+self.beta**2)*(precision*recall)/(self.beta**2*precision+recall)
#         return 1 - f_b

In [7]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [40]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'base_path': '/home/jo/Desktop/swdacon/open/', # change relative path of data
    'train_data': 'train.csv', # change train data csv name
    'test_data': 'test.csv', # change test data csv name
    'seed': 42,
    'valid_size': 0.1,
    'early_stopping': 30,
    'scheduler': True,
    'train' : {
       'batch_size' : 16,
       'num_workers': 6,
       'epochs': 1000,
       'lr': 1e-3,
    },
    'inference' : {
       'batch_size' : 16,
       'num_workers': 6,
       'threshold': 0.35,
    },
}
fix_seed(config['seed'])
custom_transform = {
    'train':A.Compose([
        A.augmentations.crops.transforms.RandomCrop(224,224,p=1.0),
        A.RandomShadow(p=0.6),
        A.OneOf(
            [
                A.RandomRotate90(p=0.5),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
            ]
        ),
        A.Normalize(),
        ToTensorV2(),
    ]),
    'valid':A.Compose([
        A.Resize(224,224),
        # A.augmentations.crops.transforms.CenterCrop(224,224,p=1.0),
        A.Normalize(),
        ToTensorV2(),
    ]),
    'test': A.Compose([
        A.Normalize(),
        ToTensorV2(),
    ]),
}

In [41]:
config['device']

device(type='cuda')

## Model

In [10]:
# from vit_pytorch.cait import CaiT
from vit_pytorch import ViT

In [16]:
# vit_encoder = DeepViT(
#     image_size=14,
#     patch_size=1,
#     dim = 128,
#     depth=12,
#     num_classes=2,
#     heads=8,
#     mlp_dim=2048,
#     emb_dropout= 0.1,
#     dropout=0.1
# )
# vit_encoder=vit_encoder.to(config['device'])

In [17]:
# vit_encoder = CaiT(
#     image_size=14,
#     patch_size=1,
#     dim = 128,
#     depth=12,
#     cls_depth=2,
#     num_classes=2,
#     heads=8,
#     mlp_dim=2048,
#     emb_dropout= 0.1,
#     dropout=0.1,
#     layer_dropout=0.05
# )
# vit_encoder=vit_encoder.to(config['device'])

In [11]:
vit_encoder = ViT(
    image_size=14,
    patch_size=1,
    dim = 128,
    depth=12,
    num_classes=2,
    heads=8,
    mlp_dim=10,
)
vit_encoder=vit_encoder.to(config['device'])

In [19]:
# print(vit_encoder)

In [42]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
class TransUnet_b5(nn.Module):
    def __init__(self):
        super(TransUnet_b5, self).__init__()
        self.backbone = models.efficientnet_b5(weights='EfficientNet_B5_Weights.DEFAULT')
        self.vit_flatten = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=1, p2=1),
            nn.LayerNorm(128)
        )
        self.conv_vit_res = nn.Sequential(
            nn.Conv2d(128,64,3,1,1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.res_conv = nn.Sequential(
            nn.Conv2d(16,16,3,1,1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )
        self.seg_conv = nn.Conv2d(16, 1, 1)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(64 + 64, 64)
        self.dconv_up2 = double_conv(40 + 64, 32)
        self.dconv_up1 = double_conv(32 + 24, 16)

        self.conv_last = nn.Conv2d(16, 1, 1)
    def forward(self, x):
        x = self.backbone.features[0](x) #48,112,112
        out1 = self.backbone.features[1](x) #24,112,112
        out2 = self.backbone.features[2](out1) #40,56,56
        out3 = self.backbone.features[3](out2) #64,28,28
        out4 = self.backbone.features[4](out3) #128,14,14

        vit_out = self.vit_flatten(out4)
        vit_out = vit_encoder.transformer(vit_out)
        vit_out = vit_out.reshape(-1,128,14,14)
        vit_out = self.conv_vit_res(vit_out) #64,14,14
        up3 = self.upsample(vit_out)#64,28,28
        up3 = torch.cat([up3, out3], dim=1) #128,28,28
        up3 = self.dconv_up3(up3) #64,28,28

        up2 = self.upsample(up3) #64,56,56
        up2 = torch.cat([up2, out2], dim=1) #40+64,56,56
        up2 = self.dconv_up2(up2) #32,56,56

        up1 = self.upsample(up2) #32,112,112
        up1 = torch.cat([up1,out1],dim=1) #32+24,112,112
        up1 = self.dconv_up1(up1) #16,112,112

        up0 = self.upsample(up1)#16,224,224
        up0 = self.res_conv(up0)
        res = self.seg_conv(up0)

        return res

In [13]:
model = TransUnet_b5().to(config['device'])
# summary(model,(3,224,224),device=config['device'].type)

In [22]:
# model = TransUnet().to(config['device'])
# summary(model,(3,224,224),device=config['device'].type)

In [23]:
# model = TransUnet_CaiT().to(config['device'])
# summary(model,(3,224,224),device=config['device'].type)

## Dataset

In [32]:
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split

In [33]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, mask_rles = None, transform=None, infer=False):
        self.img_paths = img_paths
        self.mask_rles = mask_rles
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths.iloc[idx]
        image = cv2.imread(config['base_path']+img_path[2:])
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.mask_rles.iloc[idx]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        return image, mask

In [43]:
# 로컬 사용시 활성화

train_df = pd.read_csv(f"{config['base_path']}/{config['train_data']}")
# train_df=train_df[:3000]
train, val = train_test_split(train_df, test_size=config['valid_size'], random_state=config['seed'])
print("train: ", len(train), "   valid: ", len(val))

train:  6426    valid:  714


In [44]:
train_dataset = CustomDataset(img_paths=train['img_path'], mask_rles=train['mask_rle'], transform=custom_transform['train'])
train_dataloader = DataLoader(train_dataset, batch_size=config['train']['batch_size'], shuffle=True, num_workers=config['train']['num_workers'])

valid_dataset = CustomDataset(img_paths=val['img_path'], mask_rles=val['mask_rle'], transform=custom_transform['valid'])
valid_dataloader = DataLoader(valid_dataset , batch_size=config['train']['batch_size'], shuffle=False, num_workers=config['train']['num_workers'])

## train/vali

In [28]:
# class IoULoss(nn.Module):
#     def __init__(self, weight=None, size_average=True):
#         super(IoULoss, self).__init__()
#
#     def forward(self, inputs, targets, smooth=1):
#
#         #comment out if your model contains a sigmoid or equivalent activation layer
#         inputs = F.sigmoid(inputs)
#
#         #flatten label and prediction tensors
#         inputs = inputs.view(-1)
#         targets = targets.view(-1)
#
#         #intersection is equivalent to True Positive count
#         #union is the mutually inclusive area of all labels & predictions
#         intersection = (inputs * targets).sum()
#         total = (inputs + targets).sum()
#         union = total - intersection
#
#         IoU = (intersection + smooth)/(union + smooth)
#
#         return 1 - IoU

In [18]:
def validation(config, model, valid_loader, val):
    model.eval()
    valid_loss = 0
    result = []
    transformed_mask = []
    val_df = val.copy()
    criterion_ce = torch.nn.BCEWithLogitsLoss()
    criterion_dice = DiceLoss()
    # criterion_fb = F_BetaLoss(beta=2.0)
    with torch.no_grad():
        for images, masks in tqdm(valid_loader):
            if type(transformed_mask) == torch.Tensor:
                transformed_mask = torch.cat([transformed_mask, masks])
            else:
                transformed_mask = masks.clone().detach()
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            outputs = model(images)
            # loss_ce = criterion_ce(outputs, masks.unsqueeze(1))
            loss_dice = criterion_dice(outputs,masks.unsqueeze(1))
            # loss_fb = criterion_fb(outputs,masks.unsqueeze(1))
            loss = loss_dice
            valid_loss += loss.item()

            output_masks = torch.sigmoid(outputs).cpu().numpy()
            output_masks = np.squeeze(output_masks, axis=1)
            output_masks = (output_masks > config['inference']['threshold']).astype(np.uint8)
            for i in range(len(images)):
                mask_rle = rle_encode(output_masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
        val_df['valid_mask_rle'] = result
        val_df['transformed_mask_rle'] = list(map(rle_encode, transformed_mask.squeeze().numpy()))
        dice_score = calculate_dice_scores(val_df)

    return valid_loss/len(valid_loader), dice_score

In [19]:
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts

In [20]:
def training(config, model, train_loader, valid_loader, val):
    model = model.to(config['device'])
    es_count = 0
    min_val_loss = float('inf')
    best_model = None
    criterion_ce = torch.nn.BCEWithLogitsLoss()
    criterion_dice = DiceLoss()
    # criterion_fb = F_BetaLoss(beta=2)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['train']['lr'])
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-8, verbose=True)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20,eta_min=0, verbose=True)
    scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                      first_cycle_steps=20,
                                      cycle_mult=1.0,
                                      max_lr=1e-3,
                                      min_lr=1e-6,
                                      warmup_steps=5,
                                      gamma=1.0)

    # training loop
    for epoch in range(config['train']['epochs']):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            optimizer.zero_grad()
            outputs = model(images)
            # loss_ce = criterion_ce(outputs, masks.unsqueeze(1))
            loss_dice = criterion_dice(outputs,masks.unsqueeze(1))
            loss = loss_dice
            # loss_fb = criterion_fb(outputs,masks.unsqueeze(1))
            # loss = loss_fb
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        val_loss, dice_score = validation(config, model, valid_loader, val)
        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            best_epoch = epoch + 1
            print(f"Epoch [{epoch + 1}] New Minimum Valid Loss!")

        if config['scheduler']:
            scheduler.step(val_loss)

        if es_count == config['early_stopping']:
            print(f'Epoch {epoch+1}, Train Loss(dice): {(epoch_loss/len(train_loader)):6f}, Valid Loss: {val_loss:6f}, Dice Coefficient: {dice_score:6f}, ES Count: {es_count}')
            print(f"EARLY STOPPING COUNT에 도달했습니다! \nEARLY STOPPING COUNT: {config['early_stopping']} BEST EPOCH: {best_epoch}")
            print("***TRAINING DONE***")
            return best_model

        print(f'Epoch {epoch+1}, Train Loss(dice): {(epoch_loss/len(train_loader)):6f},  Valid Loss(dice): {val_loss:6f}, Dice Coefficient: {dice_score:6f} ES Count: {es_count}')
        print("------------------------------------------------------------------------------------")

    print(f"EARLY STOPPING COUNT에 도달하지 않았습니다! \nEARLY STOPPING COUNT: {config['early_stopping']} BEST EPOCH: {best_epoch}")
    print("***TRAINING DONE***")
    return best_model

In [21]:
torch.cuda.is_available() # 학습 전에 GPU 쓰고 있나 확인

True

In [45]:
model = TransUnet_b5().to(config['device'])
model= torch.load('./open/transUnet_EfficientNet_b5_bn_all.pt')

In [46]:
a,b = validation(config,model,valid_dataloader,val)
print(f'dice_loss:{a:6f}, dice coef:{b:6f}')

100%|██████████| 45/45 [00:03<00:00, 14.33it/s]


dice_loss:0.906669, dice coef:0.057381


In [34]:
best_model = training(config, model, train_dataloader, valid_dataloader, val)

100%|██████████| 45/45 [00:03<00:00, 11.72it/s]


Epoch [1] New Minimum Valid Loss!
Epoch 1, Train Loss(dice): 0.151744,  Valid Loss(dice): 0.157410, Dice Coefficient: 0.801674 ES Count: 0
------------------------------------------------------------------------------------


100%|██████████| 45/45 [00:03<00:00, 11.64it/s]


Epoch [2] New Minimum Valid Loss!
Epoch 2, Train Loss(dice): 0.142876,  Valid Loss(dice): 0.143407, Dice Coefficient: 0.802522 ES Count: 0
------------------------------------------------------------------------------------


100%|██████████| 45/45 [00:03<00:00, 14.27it/s]


...

Epoch 196, Train Loss(dice): 0.124872,  Valid Loss(dice): 0.132633, Dice Coefficient: 0.801606 ES Count: 28
------------------------------------------------------------------------------------


100%|██████████| 45/45 [00:03<00:00, 14.80it/s]


Epoch 197, Train Loss(dice): 0.119378,  Valid Loss(dice): 0.131785, Dice Coefficient: 0.802173 ES Count: 29
------------------------------------------------------------------------------------


100%|██████████| 45/45 [00:02<00:00, 15.10it/s]


Epoch 198, Train Loss(dice): 0.122130, Valid Loss: 0.131826, Dice Coefficient: 0.801425, ES Count: 30
EARLY STOPPING COUNT에 도달했습니다! 
EARLY STOPPING COUNT: 30 BEST EPOCH: 168
***TRAINING DONE***


In [45]:
model_name = "transUnet_EfficientNet_b5_bn_all_dice.pt"

# -----------------------------kaggle-----------------------------
# torch.save(best_model, f'/kaggle/working/{model_name}')
# -----------------------------로컬-----------------------------
torch.save(best_model, f"{config['base_path']}/{model_name}")

In [46]:
# 로컬 사용시 활성화

test_df = pd.read_csv(f"{config['base_path']}/{config['test_data']}")

In [47]:
fix_seed(config['seed'])

test_dataset = CustomDataset(img_paths=test_df['img_path'], transform=custom_transform['test'], infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['inference']['batch_size'], shuffle=False, num_workers=config['inference']['num_workers'])

In [48]:
def inference(config, model, test_loader):
    with torch.no_grad():
        model.eval()
        result = []
        for images in tqdm(test_loader):
            images = images.float().to(config['device'])

            outputs = model(images)
            masks = torch.sigmoid(outputs).cpu().numpy()
            masks = np.squeeze(masks, axis=1)
            masks = (masks > config['inference']['threshold']).astype(np.uint8)

            for i in range(len(images)):
                mask_rle = rle_encode(masks[i])
                if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                    result.append(-1)
                else:
                    result.append(mask_rle)
    return result

In [49]:
inference_result = inference(config, best_model, test_dataloader)

100%|██████████| 3790/3790 [02:19<00:00, 27.07it/s]


In [40]:
# import datetime

In [50]:
# 로컬 사용시 활성화

submit = pd.read_csv(f"{config['base_path']}/sample_submission.csv")
# submit = pd.read_csv(f"{config['base_path']}/submit20.csv")
submit['mask_rle'] = inference_result

# now = datetime.datetime.now()
# current_time = now.strftime("%m%d-%H:%M:%S")
# file_name = "transUnet_EfficientNet_CaiT.csv"

submit.to_csv(f"{config['base_path']}/transUnet_EfficientNet_b5_bn_all_dice.csv", index=False)

## viewer

In [42]:
# import matplotlib.pyplot as plt

In [43]:
# def submission_viewer(test_csv, submit_csv, img_num, base_path = config['base_path'], is_colab = False):
#     """
#     white -> 건물 black -> 배경
#     1. Local에서 사용 시 test_csv, submit_csv, img_num만 입력
#     2. colab에서 사용 시 아래 사항을 입력
#     base_path = colab_base
#     is_colab = True
#     """
#     mask_rle = submit_csv.iloc[img_num, 1]
#     image_path = test_csv.iloc[img_num, 1]
#     if is_colab:
#         image = cv2.imread(image_path)
#     else:
#         image = cv2.imread(base_path + image_path[1:])
#     mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))
#     fig = plt.figure()
#     ax1 = fig.add_subplot(1,2,1)
#     ax1.imshow(image)
#     ax1.set_title('image')
#     ax2 = fig.add_subplot(1,2,2)
#     ax2.imshow(mask,cmap='gray')
#     ax2.set_title('mask')
#     plt.show()

In [44]:
# # 로컬 사용시 활성화
#
# last_submit = pd.read_csv(f"{config['base_path']}/transUnet_EfficientNet_b5_ce25iou25dice.csv")
# submission_viewer(test_df, last_submit, 0)
# submission_viewer(test_df, last_submit, 1)
# submission_viewer(test_df, last_submit, 2)